![](images/EscUpmPolit_p.gif "UPM")

# Course Notes for Learning Intelligent Systems

Department of Telematic Engineering Systems, Universidad Politécnica de Madrid, © 2016 Carlos A. Iglesias

# Text Classification

# Table of Contents
* [Objectives](#Objectives)
* [Corpus](#Corpus)
* [Classifier](#Classifier)

# Objectives

In this session we provide a quick overview of how the vector models we have presented previously can be used for applying machine learning techniques, such as classification.

The main objectives of this session are:
* Understand how to apply machine learning techniques on textual sources
* Learn the facilities provided by scikit-learn

# Corpus

We are going to use on of the corpus that come prepackaged with Scikit-learn: the [20 newsgroup datase](http://qwone.com/~jason/20Newsgroups/). The 20  newsgroup dataset contains 20k documents that belong to 20 topics.

We inspect now the corpus using the facilities from Scikit-learn, as explain in [scikit-learn](http://scikit-learn.org/stable/datasets/twenty_newsgroups.html#newsgroups)

In [1]:
from sklearn.datasets import fetch_20newsgroups

# We remove metadata to avoid bias in the classification
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

# print categories
print(list(newsgroups_train.target_names))

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [2]:
#Number of categories
print(len(newsgroups_train.target_names))

20


In [3]:
# Show a document
docid = 1
doc = newsgroups_train.data[docid]
cat = newsgroups_train.target[docid]

print("Category id " +  str(cat) + " " + newsgroups_train.target_names[cat])
print("Doc " + doc)

Category id 4 comp.sys.mac.hardware
Doc A fair number of brave souls who upgraded their SI clock oscillator have
shared their experiences for this poll. Please send a brief message detailing
your experiences with the procedure. Top speed attained, CPU rated speed,
add on cards and adapters, heat sinks, hour of usage per day, floppy disk
functionality with 800 and 1.4 m floppies are especially requested.

I will be summarizing in the next two days, so please add to the network
knowledge base if you have done the clock upgrade and haven't answered this
poll. Thanks.


In [4]:
#Number of files
newsgroups_train.filenames.shape

(11314,)

In [5]:
# Obtain a vector

from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(analyzer='word', stop_words='english')

vectors_train = vectorizer.fit_transform(newsgroups_train.data)
vectors_train.shape

/home/cif/anaconda3/lib/python3.5/site-packages/numpy/core/fromnumeric.py:2652: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  VisibleDeprecationWarning)


(11314, 101323)

In [6]:
# The tf-idf vectors are very sparse with an average of 66 non zero components in 101.323 dimensions (.06%)
vectors_train.nnz / float(vectors_train.shape[0])

66.80510871486653

# Classifier

Once we have vectors, we can create classifiers (or other machine learning algorithms such as clustering) as we saw previously in the notebooks of machine learning with scikit-learn.

In [7]:
from sklearn.naive_bayes import MultinomialNB

from sklearn import metrics


# We learn the vocabulary (fit) with the train dataset and transform into vectors (fit_transform)
# Nevertheless, we only transform the test dataset into vectors  (transform, not fit_transform)

model = MultinomialNB(alpha=.01)
model.fit(vectors_train, newsgroups_train.target)

vectors_test = vectorizer.transform(newsgroups_test.data)
pred = model.predict(vectors_test)

metrics.f1_score(newsgroups_test.target, pred, average='weighted')


/home/cif/anaconda3/lib/python3.5/site-packages/numpy/core/fromnumeric.py:2652: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  VisibleDeprecationWarning)


0.69545360719001303

We are getting F1 of 0.69 for 20 categories this could be improved (optimization, preprocessing, etc.)

In [8]:
from sklearn.utils.extmath import density

print("dimensionality: %d" % model.coef_.shape[1])
print("density: %f" % density(model.coef_))

dimensionality: 101323
density: 1.000000


In [9]:
# We can review the top features per topic in Bayes (attribute coef_)
import numpy as np

def show_top10(classifier, vectorizer, categories):
    feature_names = np.asarray(vectorizer.get_feature_names())
    for i, category in enumerate(categories):
        top10 = np.argsort(classifier.coef_[i])[-10:]
        print("%s: %s" % (category, " ".join(feature_names[top10])))

        
show_top10(model, vectorizer, newsgroups_train.target_names)

alt.atheism: islam atheists say just religion atheism think don people god
comp.graphics: looking format 3d know program file files thanks image graphics
comp.os.ms-windows.misc: card problem thanks driver drivers use files dos file windows
comp.sys.ibm.pc.hardware: monitor disk thanks pc ide controller bus card scsi drive
comp.sys.mac.hardware: know monitor does quadra simms thanks problem drive apple mac
comp.windows.x: using windows x11r5 use application thanks widget server motif window
misc.forsale: asking email sell price condition new shipping offer 00 sale
rec.autos: don ford new good dealer just engine like cars car
rec.motorcycles: don just helmet riding like motorcycle ride bikes dod bike
rec.sport.baseball: braves players pitching hit runs games game baseball team year
rec.sport.hockey: league year nhl games season players play hockey team game
sci.crypt: people use escrow nsa keys government chip clipper encryption key
sci.electronics: don thanks voltage used know does lik

In [10]:
# We try the classifier in two new docs

new_docs = ['This is a survey of PC computers', 'God is love']
new_vectors = vectorizer.transform(new_docs)

pred_docs = model.predict(new_vectors)
print(pred_docs)
print([newsgroups_train.target_names[i] for i in pred_docs])

[ 2 15]
['comp.os.ms-windows.misc', 'soc.religion.christian']


/home/cif/anaconda3/lib/python3.5/site-packages/numpy/core/fromnumeric.py:2652: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  VisibleDeprecationWarning)


## References



* [NLTK Book. Natural Language Processing with Python. Steven Bird, Ewan Klein, and Edward Loper. O'Reilly Media, 2009 ](http://www.nltk.org/book_1ed/)
* [NLTK Essentials, Nitin Hardeniya, Packt Publishing, 2015](http://proquest.safaribooksonline.com/search?q=NLTK%20Essentials)

## Licence

The notebook is freely licensed under under the [Creative Commons Attribution Share-Alike license](https://creativecommons.org/licenses/by/2.0/).  

© 2016 Carlos A. Iglesias, Universidad Politécnica de Madrid.